In [1]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
target = '标签'
uid = '申请编号'
columns = {}
columns['no_fea'] = [uid, target, '申请时点']
columns['01'] = [
    '贷款类型',
    '是否提供手机号',
    '是否提供电话',
    '手机号是否有效',
    '是否提供email',
    '申请人是否额外提供了文件2',
    '申请人是否额外提供了文件3',
    '申请人是否额外提供了文件4',
    '申请人是否额外提供了文件5',
    '申请人是否额外提供了文件6',
    '申请人是否额外提供了文件7',
    '申请人是否额外提供了文件8',
    '申请人是否额外提供了文件9',
    '是否有车',
    '是否有房',
    '地址是否一致标志1',
    '地址是否一致标志2',
    '地址是否一致标志3',
    '地址是否一致标志4',
    '地址是否一致标志5',
    '地址是否一致标志6']
columns['100'] = [
    '性别',
    '客户居住地评分1',
    '客户居住地评分2',
    '教育程度',
    '婚姻状态',
    '居住状态',
    '贷款申请前1小时内征信查询次数',
    '贷款申请前1天内征信查询次数',
    '贷款申请前1周内征信查询次数',
    '贷款申请前1个月内征信查询次数',
    '贷款申请前1个季度内征信查询次数',
    '贷款申请前1年内征信查询次数',
    '收入类型',
    '陪同申请人',
    '孩子个数',
    '职业',
    '家庭成员数',
    '社交圈违约信息1_1',
    '社交圈违约信息2_1',
    '有车时间',
    '单位类型']
columns['time'] = [
    '申请周内日',
    '申请时点',
    '出生日期距申请日期天数']
columns['_dt'] = [
    '最近一次换手机号码距申请日天数',
    '出生日期距申请日期天数',
    '工作日期距申请日期天数',
    '注册日期距申请日期天数',
    '身份认证日期距申请日期天数']
columns['money'] = [
    '贷款年金',
    '客户收入',
    '商品价格',
    '信用额度']

In [5]:
############## Functions

def ana_dtypes(t):
    if t == 'str':
        cols = types[types == 'object'].index
    else:
        cols = types[types != 'object'].index
    # Save description
    cols_desc = d[cols].describe().T
    cols_desc.to_csv(f'./data/cols_desc_{t}.csv')
    # Return columns
    return cols

# a = d11.select_dtypes(include=['int64'])

In [6]:
############## Load

d0 = pd.read_csv('./data/train_label.csv')
d0.shape
d0.head()

# Target Count
d0['标签'].value_counts()

(140000, 2)

,申请编号,标签
0,0,1
1,1,0
2,2,0
3,3,0
4,5,0


0    119000
1     21000
Name: 标签, dtype: int64

In [7]:
d1 = pd.read_csv('./data/train_Application.csv')
d1.shape
d1.head()
d1.dtypes.value_counts()

(140000, 31)

,申请编号,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,身份认证日期距申请日期天数,...,申请人是否额外提供了文件6,申请人是否额外提供了文件7,申请人是否额外提供了文件8,申请人是否额外提供了文件9,贷款申请前1小时内征信查询次数,贷款申请前1天内征信查询次数,贷款申请前1周内征信查询次数,贷款申请前1个月内征信查询次数,贷款申请前1个季度内征信查询次数,贷款申请前1年内征信查询次数
0,0,0,460190.889355,17463.042019,419951.511045,7.0,-10107,-342,-5421,-3292,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,424370.659603,15585.046388,384597.975692,7.0,-13980,-1110,-3387,-826,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
2,2,0,469330.587153,18837.558252,419951.511045,1.0,-13331,-2246,-3870,-171,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
3,3,0,464188.353140,16754.587069,409850.500944,7.0,-16540,292204,-970,-2916,...,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0,434196.988738,16165.009200,394698.985793,7.0,-17919,-11037,-9350,-3588,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0


int64      20
float64    11
dtype: int64

In [8]:
d2 = pd.read_csv('./data/train_Personas.csv')
d2.shape
d2.dtypes.value_counts()
d2.head()


(140000, 32)

int64      18
float64    14
dtype: int64

,申请编号,性别,是否有车,是否有房,孩子个数,客户收入,收入类型,教育程度,婚姻状态,居住状态,...,地址是否一致标志3,地址是否一致标志4,地址是否一致标志5,地址是否一致标志6,单位类型,社交圈违约信息2_2,社交圈违约信息2_1,社交圈违约信息1_2,社交圈违约信息1_1,最近一次换手机号码距申请日天数
0,0,1,0,0,0,237349.580536,1,2,3,1,...,0,0,0,0,5,0.0,0.0,0.0,0.0,-887.0
1,1,1,0,1,0,237236.363407,7,4,1,1,...,0,0,0,1,5,0.0,0.0,0.0,0.0,-271.0
2,2,0,1,1,-1,237321.276253,1,4,1,1,...,1,0,0,0,5,0.0,0.0,0.0,0.0,-332.0
3,3,0,0,1,-1,237304.293684,3,4,2,1,...,0,0,0,0,57,1.0,0.0,1.0,0.0,-204.0
4,5,0,0,0,-1,237321.276253,1,1,2,1,...,1,1,0,1,53,0.0,0.0,0.0,0.0,-1049.0


In [116]:
### Merge Application

d11 = pd.merge(d0, d1, how='inner', on='申请编号')
d11.shape
d11.head()

d11 = pd.merge(d11, d2, how='inner', on='申请编号')
d11.shape
d11.head()

d11.to_csv(f'./tmp/0_merge11.csv')

(140000, 32)

,申请编号,标签,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,...,申请人是否额外提供了文件6,申请人是否额外提供了文件7,申请人是否额外提供了文件8,申请人是否额外提供了文件9,贷款申请前1小时内征信查询次数,贷款申请前1天内征信查询次数,贷款申请前1周内征信查询次数,贷款申请前1个月内征信查询次数,贷款申请前1个季度内征信查询次数,贷款申请前1年内征信查询次数
0,0,1,0,460190.889355,17463.042019,419951.511045,7.0,-10107,-342,-5421,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,0,424370.659603,15585.046388,384597.975692,7.0,-13980,-1110,-3387,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
2,2,0,0,469330.587153,18837.558252,419951.511045,1.0,-13331,-2246,-3870,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
3,3,0,0,464188.353140,16754.587069,409850.500944,7.0,-16540,292204,-970,...,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0,0,434196.988738,16165.009200,394698.985793,7.0,-17919,-11037,-9350,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0


(140000, 63)

,申请编号,标签,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,...,地址是否一致标志3,地址是否一致标志4,地址是否一致标志5,地址是否一致标志6,单位类型,社交圈违约信息2_2,社交圈违约信息2_1,社交圈违约信息1_2,社交圈违约信息1_1,最近一次换手机号码距申请日天数
0,0,1,0,460190.889355,17463.042019,419951.511045,7.0,-10107,-342,-5421,...,0,0,0,0,5,0.0,0.0,0.0,0.0,-887.0
1,1,0,0,424370.659603,15585.046388,384597.975692,7.0,-13980,-1110,-3387,...,0,0,0,1,5,0.0,0.0,0.0,0.0,-271.0
2,2,0,0,469330.587153,18837.558252,419951.511045,1.0,-13331,-2246,-3870,...,1,0,0,0,5,0.0,0.0,0.0,0.0,-332.0
3,3,0,0,464188.353140,16754.587069,409850.500944,7.0,-16540,292204,-970,...,0,0,0,0,57,1.0,0.0,1.0,0.0,-204.0
4,5,0,0,434196.988738,16165.009200,394698.985793,7.0,-17919,-11037,-9350,...,1,1,0,1,53,0.0,0.0,0.0,0.0,-1049.0


In [119]:
d11.groupby(['标签', '客户居住地评分1']).agg({'申请编号':'count', '标签':'mean' })

申请编号  标签
标签 客户居住地评分1           
0  0         12846   0
   1         88061   0
   2         18093   0
1  0          1316   1
   1         15152   1
   2          4532   1

In [10]:
''' 重复？'''
d11[uid].nunique()

''' Target '''
d11[target].value_counts()

''' Dtypes '''
d11.dtypes.value_counts()

''' Description '''
d_feats = d11.drop([uid, target], axis=1)
cols_desc = d_feats.describe().T
# Unique
cols_unique = d_feats.apply(pd.Series.nunique, axis = 0)
cols_unique = pd.DataFrame(cols_unique, columns=['unique'])
cols_desc = cols_desc.join(cols_unique)

cols_desc.head()
cols_desc.to_csv(f'./tmp/0_cols_desc.csv')

' 重复？'

140000

' Target '

0    119000
1     21000
Name: 标签, dtype: int64

' Dtypes '

int64      38
float64    25
dtype: int64

' Description '

,count,mean,std,min,25%,50%,75%,max,unique
贷款类型,140000.0,0.092671,0.289972,0.000000,0.000000,0.000000,0.000000,1.000000,2
信用额度,140000.0,478907.188397,51221.158226,408260.788232,437110.844412,468139.656834,506177.878906,921791.788232,4516
贷款年金,139993.0,17123.620552,1401.602040,14650.637578,16103.385420,16914.983717,17852.888912,36847.435823,11666
商品价格,139880.0,429468.479262,41142.193912,374496.965591,396214.137308,419951.511045,445709.086803,823991.915085,714
陪同申请人,139430.0,6.082084,2.116810,0.000000,7.000000,7.000000,7.000000,7.000000,7


In [11]:
############## Outlier：Get

low = .01
high = .99
quant_df = d11.drop([uid, target], axis=1).quantile([low, high]).T
quant_df.to_csv('./tmp/0_outlier.csv')

In [12]:
# 处理
d12 = d11.copy()

# 置空值
d12.loc[d12['孩子个数']==-1, '孩子个数'] = np.NaN
# 距申请日期>0
for col in columns['_dt']:
    d12.loc[d12[col]>0, col] = np.NaN

In [14]:
############## Outlier：Apply

outliers = pd.read_csv('./tmp/0_outlier1.csv', index_col=0)
outliers.shape
outliers.head()

def set_outlier(col):
    if col < col_min:
        col = col_min
    elif col > col_max:
        col = col_max
    return col

# def set_outlier(col):
#     if col < col_min or col > col_max:
#         col = np.NaN
#     return col
    
for col in outliers.index:
# for col in '出生日期距申请日期天数', '社交圈违约信息1_2':
    col_min = outliers.loc[col, 'min']
    col_max = outliers.loc[col, 'max']
    d12[col] = d12[col].apply(set_outlier)
    
d12.to_csv(f'./tmp/2_eda_outlier.csv')

(61, 2)

,min,max
地址是否一致标志1,0,1
地址是否一致标志2,0,1
地址是否一致标志3,0,1
地址是否一致标志4,0,1
地址是否一致标志5,0,1


In [15]:
############# correlations

correlations = d12.drop([uid], axis=1).corr()
correlations.to_csv('./tmp/0_correlations.csv')

In [16]:
d12.loc[d12['出生日期距申请日期天数'], '身份认证日期距申请日期天数'].min()

KeyError: "None of [Float64Index([-10107.0, -13980.0, -13331.0, -16540.0, -17919.0,  -9217.0,\n               -8174.0, -14511.0, -12440.0, -18104.0,\n              ...\n              -15130.0, -12726.0, -13812.0,  -9276.0, -11029.0,  -7997.0,\n              -18731.0, -10755.0, -11036.0,  -7782.0],\n             dtype='float64', length=140000)] are in the [index]"

In [ ]:
for col in ['社交圈违约信息1_2']:
    d12[col].value_counts()

In [ ]:
############## Feature Engineering

### Correlation

def calc_poly(cols):
    degree = 2
    poly_features = d12[cols]
    poly_target = d12[target]

    # Fill Miss
    from sklearn.preprocessing import Imputer
    imputer = Imputer(strategy = 'median')
    poly_features = imputer.fit_transform(poly_features)

    # Polynomial
    from sklearn.preprocessing import PolynomialFeatures
    poly_transformer = PolynomialFeatures(degree = degree, include_bias=False) # interaction_only
    poly_transformer.fit(poly_features)
    poly_features = poly_transformer.transform(poly_features)
    poly_features = pd.DataFrame(poly_features, 
                                 columns = poly_transformer.get_feature_names(cols))
    return poly_features


cols = ['出生日期距申请日期天数',
        '工作日期距申请日期天数',
        '最近一次换手机号码距申请日天数',
        '客户居住地评分2',
        '客户居住地评分1',
        '教育程度',
        '性别',
        '身份认证日期距申请日期天数',
        '地址是否一致标志6',
        '收入类型',
        '是否提供手机号',
        '地址是否一致标志2',
        '注册日期距申请日期天数',
        '有车时间']
#         '居住状态',
#         '地址是否一致标志4',
#         '是否提供电话',
#         '职业',
#         '社交圈违约信息2_1',
#         '贷款申请前1年内征信查询次数',
#         '社交圈违约信息1_1']
poly_features = calc_poly(cols)
poly_features.shape
poly_features.head()

In [283]:
d13 = pd.concat([d12, poly_features.drop(cols, axis=1)], axis=1)
d13.shape
d13.head()

d13.to_csv(f'./tmp/3_poly.csv')

(140000, 168)

,申请编号,标签,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,...,是否提供手机号^2,是否提供手机号 地址是否一致标志2,是否提供手机号 注册日期距申请日期天数,是否提供手机号 有车时间,地址是否一致标志2^2,地址是否一致标志2 注册日期距申请日期天数,地址是否一致标志2 有车时间,注册日期距申请日期天数^2,注册日期距申请日期天数 有车时间,有车时间^2
0,0,1,0,460190.889355,17463.042019,419951.511045,7.0,-10107.0,-342.0,-5421.0,...,1.0,0.0,-5421.0,4.0,0.0,-0.0,0.0,29387241.0,-21684.0,16.0
1,1,0,0,424370.659603,15585.046388,384597.975692,7.0,-13980.0,-1110.0,-3387.0,...,1.0,1.0,-3387.0,4.0,1.0,-3387.0,4.0,11471769.0,-13548.0,16.0
2,2,0,0,469330.587153,18837.558252,419951.511045,1.0,-13331.0,-2246.0,-3870.0,...,1.0,0.0,-3870.0,1.0,0.0,-0.0,0.0,14976900.0,-3870.0,1.0
3,3,0,0,464188.353140,16754.587069,409850.500944,7.0,-16540.0,NaN,-970.0,...,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,940900.0,-3880.0,16.0
4,5,0,0,434196.988738,16165.009200,394698.985793,7.0,-17919.0,-11037.0,-9350.0,...,1.0,0.0,-9350.0,4.0,0.0,-0.0,0.0,87422500.0,-37400.0,16.0


In [74]:
### 导入d3历史申请表数据 ###
d3 = pd.read_csv('./data/train_History_Application.csv')
d3.shape
d3.dtypes
d3.head()

(647963, 32)

历史申请编号             int64
申请编号               int64
贷款类型              object
贷款年金             float64
申请额度               int64
信用额度             float64
首付金额             float64
商品价格             float64
申请周内日              int64
申请时点               int64
是否为最后一次申请记录       object
是否为当天最后一次申请记录      int64
首付率              float64
利率1              float64
利率2              float64
贷款用途               int64
合同状态               int64
历史贷款授信距本次申请时间      int64
付款方式               int64
拒绝原因               int64
陪同申请人            float64
客户类型               int64
商品类别               int64
组合类型               int64
产品类型               int64
获客渠道               int64
销售区域               int64
行业                 int64
贷款期数             float64
收益级别               int64
产品组合             float64
首付时间             float64
dtype: object

,历史申请编号,申请编号,贷款类型,贷款年金,申请额度,信用额度,首付金额,商品价格,申请周内日,申请时点,...,商品类别,组合类型,产品类型,获客渠道,销售区域,行业,贷款期数,收益级别,产品组合,首付时间
0,1040733,173562,0,4444.0,106931,119633.0,NaN,106931.0,5,5,...,27,2,2,3,0,10,44.0,2,7.0,292204.0
1,450011,173562,0,3831.0,79211,90188.0,NaN,79211.0,7,4,...,27,2,2,0,4,10,44.0,3,7.0,292204.0
2,698695,173562,A,861.0,8327,4180.0,4367.0,8327.0,5,5,...,2,3,0,7,84,4,7.0,1,10.0,292204.0
3,924081,173562,A,1183.0,12568,12568.0,11.0,12568.0,7,4,...,7,3,0,7,83,5,14.0,3,12.0,292204.0
4,907875,173562,1,1990.0,11,39611.0,NaN,NaN,6,6,...,27,0,2,3,0,10,0.0,0,1.0,-560.0


In [79]:
### 特征分类 ###
target = '标签'
uid = '申请编号'
hid = '历史申请编号'
col_cat = {}
col_cat['no_fea'] = [uid, target, hid]
# d3_feats = d3.drop([uid, hid], axis=1)

# 金额型
col_cat['curr'] = [
    '贷款年金',
    '申请额度',
    '信用额度',
    '首付金额',
    '商品价格']

# 类别型
col_cat['cat'] = [
    '贷款类型',
    '贷款用途',
    '合同状态',
    '付款方式',
    '拒绝原因',
    '陪同申请人',
    '客户类型',
    '商品类别',
    '组合类型',
    '产品类型',
    '获客渠道',
    '销售区域',
    '行业',
    '收益级别',
    '产品组合']

# 时点型
col_cat['time_point'] = [
    '申请周内日',
    '申请时点']

# 时长型
col_cat['time_span'] = [
    '历史贷款授信距本次申请时间',
    '首付时间']

# 布尔型
col_cat['bool'] = [
    '是否为最后一次申请记录',
    '是否为当天最后一次申请记录']

# 概率型
col_cat['prob'] = [
    '首付率',
    '利率1',
    '利率2']

# 计数型
col_cat['cnt'] = [
    '贷款期数']

In [80]:
### 数据清洗 - 金额型 ###
# col_cat['curr'] = [
#     '贷款年金',
#     '申请额度',
#     '信用额度',
#     '首付金额',
#     '商品价格']

# ''' 查看含有缺失值字段 '''
# for col in col_cat['curr']:
#     d3[col].value_counts(dropna=False)

# ''' 含有缺失值字段 '''
# nan_cols = [
#     '贷款年金',
#     '首付金额',
#     '商品价格'
# ]

# 数值型缺失插补均值
d3.fillna({
    '贷款年金':d3['贷款年金'].mean(),
    '首付金额':d3['首付金额'].mean(),
    '商品价格':d3['商品价格'].mean()}
     ,inplace=True)

In [81]:
### 数据清洗 - 时长型 ###
# col_cat['time_span'] = [
#     '历史贷款授信距本次申请时间',
#     '首付时间']

# ''' 查看含有缺失值字段 '''
# for col in col_cat['time_span']:
#     d3[col].value_counts(dropna=False)

# 数值型异常值置None
d3.loc[
    (d3['首付时间'] == 292204.0), '首付时间'] = None

# 数值型缺失均值
d3.fillna({
    '首付时间':d3['首付时间'].mean()}
     ,inplace=True)

In [82]:
### 数据清洗 - 概率型 ###
# col_cat['prob'] = [
#     '首付率',
#     '利率1',
#     '利率2']

# ''' 查看含有缺失值字段 '''
# for col in col_cat['prob']:
#     d3[col].value_counts(dropna=False)
    
# 数值型缺失均值
d3.fillna({
    '首付率':d3['首付率'].mean(),
    '利率1':d3['利率1'].mean(),
    '利率2':d3['利率2'].mean()}
     ,inplace=True)

In [83]:
### 数据清洗 - 类别型 ###
# col_cat['cat'] = [
#     '贷款类型',
#     '贷款用途',
#     '合同状态',
#     '付款方式',
#     '拒绝原因',
#     '陪同申请人',
#     '客户类型',
#     '商品类别',
#     '组合类型',
#     '产品类型',
#     '获客渠道',
#     '销售区域',
#     '行业',
#     '收益级别',
#     '产品组合']

# ''' 查看含有缺失值字段 '''
# for col in col_cat['cat']:
#     d3[col].value_counts(dropna=False)

# ''' d3 - 类别型desc '''
# d3_cat_desc = d3_feats[col_cat['cat']].astype("object").describe(include=['object'])
# print(d3_cat_desc)

# ''' 含有缺失值字段 '''
# nan_cols = [
#     '贷款类型',
#     '陪同申请人',
#     '产品组合'
# ]

### 处理含空值的离散特征，转换dummies

# d3['贷款类型'].value_counts()
# d3['陪同申请人'].value_counts()
# d3['产品组合'].value_counts()

d3_dum = pd.get_dummies(d3,dummy_na=True,columns=[
    '贷款类型',
    '陪同申请人',
    '产品组合'
])

# 空值取众数
d3_dum.loc[d3_dum['贷款类型_nan']==1, '贷款类型_0'] = 1
d3_dum.loc[d3_dum['陪同申请人_nan']==1, '陪同申请人_7.0'] = 1
d3_dum.loc[d3_dum['产品组合_nan']==1, '产品组合_2.0'] = 1

### 处理其他离散特征，有长尾分布, 小值归other类，转换dummies

# d3['贷款用途'].value_counts()

d3_dum.loc[(d3_dum['贷款用途'] != 23) & (d3_dum['贷款用途'] != 24), '贷款用途'] = 'other'

# d3['贷款用途'].value_counts()
# d3['合同状态'].value_counts()
# d3['付款方式'].value_counts()
# d3['拒绝原因'].value_counts()
# d3['客户类型'].value_counts()
# d3['商品类别'].value_counts()

d3_dum.loc[
    (d3_dum['商品类别'] != 27) & 
    (d3_dum['商品类别'] != 19) &
    (d3_dum['商品类别'] != 7) &
    (d3_dum['商品类别'] != 5) &
    (d3_dum['商品类别'] != 2) &
    (d3_dum['商品类别'] != 11), '商品类别'] = 'other'

# d3['组合类型'].value_counts()
# d3['产品类型'].value_counts()
# d3['获客渠道'].value_counts()
# d3['销售区域'].value_counts()

d3_dum['销售区域_count'] = d3.groupby('销售区域')['销售区域'].transform('count')
d3_dum.loc[d3_dum['销售区域_count'] < 5000, '销售区域'] = 'other'
# d3 = d3.drop('销售区域_count', axis=1)

# d3['行业'].value_counts()

d3_dum = pd.get_dummies(d3_dum, columns=[
    '贷款用途',
    '合同状态',
    '付款方式',
    '拒绝原因',
    '客户类型',
    '商品类别',
    '组合类型',
    '产品类型',
    '获客渠道',
    '销售区域',
    '行业'
])

In [84]:
### 数据清洗 - 时点型 ###

# 时点型(类别)
# col_cat['time_point'] = [
#     '申请周内日',
#     '申请时点']

# ''' 查看含有缺失值字段 '''
# for col in col_cat['time_point']:
#     d3[col].value_counts(dropna=False)

### 处理含空值的离散特征，转换dummies

# d3['申请周内日'].value_counts()
# d3['申请时点'].value_counts()

d3_dum.loc[
    (d3_dum['申请时点'] != 3) & 
    (d3_dum['申请时点'] != 4) &
    (d3_dum['申请时点'] != 5) &
    (d3_dum['申请时点'] != 6) &
    (d3_dum['申请时点'] != 7) &
    (d3_dum['申请时点'] != 8) &
    (d3_dum['申请时点'] != 9), '申请时点'] = 'other'

d3_dum = pd.get_dummies(d3_dum,columns=[
    '申请周内日',
    '申请时点'
])

In [85]:
### check ###

d3_dum.shape
for col in d3_dum.columns:
    print(col)
d3_dum.head()

(647963, 134)

历史申请编号
申请编号
贷款年金
申请额度
信用额度
首付金额
商品价格
是否为最后一次申请记录
是否为当天最后一次申请记录
首付率
利率1
利率2
历史贷款授信距本次申请时间
贷款期数
收益级别
首付时间
贷款类型_0
贷款类型_1
贷款类型_A
贷款类型_nan
陪同申请人_0.0
陪同申请人_1.0
陪同申请人_2.0
陪同申请人_4.0
陪同申请人_5.0
陪同申请人_6.0
陪同申请人_7.0
陪同申请人_nan
产品组合_0.0
产品组合_1.0
产品组合_2.0
产品组合_3.0
产品组合_4.0
产品组合_5.0
产品组合_6.0
产品组合_7.0
产品组合_8.0
产品组合_10.0
产品组合_11.0
产品组合_12.0
产品组合_13.0
产品组合_14.0
产品组合_15.0
产品组合_16.0
产品组合_17.0
产品组合_nan
销售区域_count
贷款用途_23
贷款用途_24
贷款用途_other
合同状态_0
合同状态_1
合同状态_2
合同状态_3
付款方式_0
付款方式_1
付款方式_2
付款方式_3
拒绝原因_0
拒绝原因_1
拒绝原因_2
拒绝原因_3
拒绝原因_4
拒绝原因_5
拒绝原因_6
拒绝原因_7
拒绝原因_8
客户类型_0
客户类型_1
客户类型_2
客户类型_3
商品类别_2
商品类别_5
商品类别_7
商品类别_11
商品类别_19
商品类别_27
商品类别_other
组合类型_0
组合类型_1
组合类型_2
组合类型_3
组合类型_4
产品类型_0
产品类型_1
产品类型_2
获客渠道_0
获客渠道_1
获客渠道_2
获客渠道_3
获客渠道_4
获客渠道_5
获客渠道_6
获客渠道_7
销售区域_0
销售区域_3
销售区域_7
销售区域_11
销售区域_15
销售区域_19
销售区域_23
销售区域_26
销售区域_30
销售区域_38
销售区域_77
销售区域_115
销售区域_154
销售区域_other
行业_0
行业_1
行业_2
行业_3
行业_4
行业_5
行业_6
行业_7
行业_8
行业_9
行业_10
申请周内日_0
申请周内日_1
申请周内日_3
申请周内日_4
申请周内日_5
申请周内日_6
申请周内日_7
申请时点_3
申请时点_4
申请时点_5
申请时点_6
申请时点_7
申请

,历史申请编号,申请编号,贷款年金,申请额度,信用额度,首付金额,商品价格,是否为最后一次申请记录,是否为当天最后一次申请记录,首付率,...,申请周内日_6,申请周内日_7,申请时点_3,申请时点_4,申请时点_5,申请时点_6,申请时点_7,申请时点_8,申请时点_9,申请时点_other
0,1040733,173562,4444.0,106931,119633.0,1162.017652,106931.000000,Y,1,0.079750,...,0,0,0,0,1,0,0,0,0,0
1,450011,173562,3831.0,79211,90188.0,1162.017652,79211.000000,Y,1,0.079750,...,0,1,0,1,0,0,0,0,0,0
2,698695,173562,861.0,8327,4180.0,4367.000000,8327.000000,Y,1,0.556485,...,0,0,0,0,1,0,0,0,0,0
3,924081,173562,1183.0,12568,12568.0,11.000000,12568.000000,Y,1,0.000000,...,0,1,0,1,0,0,0,0,0,0
4,907875,173562,1990.0,11,39611.0,1162.017652,39791.474115,Y,1,0.079750,...,1,0,0,0,0,1,0,0,0,0


In [112]:
### d3 group by ###

agg_dict = {}

# 数值型 - 金额
curr_agg = ['mean', 'max', 'min']
for col in col_cat['curr']:
    agg_dict[col] = curr_agg

# 数值型 - 时长 
time_span_agg = ['mean', 'max', 'min']
for col in col_cat['time_span']:
    agg_dict[col] = time_span_agg

# 数值型 - 概率
prob_agg = ['mean', 'max', 'min']
for col in col_cat['prob']:
    agg_dict[col] = prob_agg

# 离散型 - 类型
cat_agg = ['sum', 'mean']
for col1 in d3_dum.columns:
    for col2 in col_cat['cat']:
        if (col2 in col1):
            agg_dict[col1] = cat_agg

# 离散型 - 时点
time_point_agg = ['sum', 'mean']
for col1 in d3_dum.columns:
    for col2 in col_cat['time_point']:
        if (col2 in col1):
            agg_dict[col1] = time_point_agg



            

# print(agg_dict)

d30 = d3_dum.groupby('申请编号').agg(agg_dict)





In [114]:
d30.columns = pd.Series(d30.columns.tolist()).apply(pd.Series).sum(axis=1)
# d30.head

In [115]:
### Merge Application
d12_d30 = pd.merge(d12, d30, how='left', on='申请编号')
d12_d30.shape
d12_d30.head()

d12_d30.to_csv(f'./tmp/d12_d30.csv')

(140000, 331)

,申请编号,标签,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,...,申请时点_6sum,申请时点_6mean,申请时点_7sum,申请时点_7mean,申请时点_8sum,申请时点_8mean,申请时点_9sum,申请时点_9mean,申请时点_othersum,申请时点_othermean
0,0,1,0,460190.889355,17463.042019,419951.511045,7.0,-10107.0,-342.0,-5421.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,1,0,0,424370.659603,15585.046388,384597.975692,7.0,-13980.0,-1110.0,-3387.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
2,2,0,0,469330.587153,18837.558252,419951.511045,1.0,-13331.0,-2246.0,-3870.0,...,1.0,0.1,7.0,0.7,1.0,0.1,1.0,0.1,0.0,0.0
3,3,0,0,464188.353140,16754.587069,409850.500944,7.0,-16540.0,NaN,-970.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0,434196.988738,16165.009200,394698.985793,7.0,-17919.0,-11037.0,-9350.0,...,1.0,0.2,0.0,0.0,2.0,0.4,1.0,0.2,0.0,0.0
